In [16]:
#--- IMPORT DEPENDENCIES ------------------------------------------------------+
# from __future__ import division
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import animation
import os
import random
import math
import pandas as pd
global  columns,iterationanimation,particlelocationsanimation,erroranimation
columns=['Iteration','ParticleLocations','FunctionCost']

In [17]:
#--- COST FUNCTION ------------------------------------------------------------+

# function we are attempting to optimize (minimize)
#ASSUMING a = 0 and b = 100 for Rosenbrock function to give global minima at (0,0)
def func1(x):
    return (-x[0])**2 + 100*(x[1]-x[0]**2)**2

In [18]:

#STEP 1 --> defining and initiating the charactertic features of a single particle
#STEP 2 --> Defining the functions to modify the particle's location and velocity
class Particle:
    def __init__(self,startlocation):
        self.loc_particle=[]          # particle position
        self.velocity_particle=[]          # particle velocity
        self.loc_best_particle=[]          # best position individual
        self.error_best_particle=-1          # best error individual
        self.error_particle=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_particle.append(random.uniform(-1,1))
            self.loc_particle.append(startlocation[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.error_particle=costFunc(self.loc_particle)

        # check to see if the current position is an individual best
        if self.error_particle < self.error_best_particle or self.error_best_particle==-1:
            self.loc_best_particle=self.loc_particle
            self.error_best_particle=self.error_particle

    # update new particle velocity
    def update_velocity(self,pos_best_swarm):
        w=0.7       # constant inertia weight (how much to weigh the previous velocity)
        c1=2        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.loc_best_particle[i]-self.loc_particle[i])
            vel_social=c2*r2*(pos_best_swarm[i]-self.loc_particle[i])
            self.velocity_particle[i]=w*self.velocity_particle[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,dimensionboundaries):
        for i in range(0,num_dimensions):
            self.loc_particle[i]=self.loc_particle[i]+self.velocity_particle[i]

            # adjust maximum position if necessary
            if self.loc_particle[i]>dimensionboundaries[i][1]:
                self.loc_particle[i]=dimensionboundaries[i][1]

            # adjust minimum position if neseccary
            if self.loc_particle[i] < dimensionboundaries[i][0]:
                self.loc_particle[i]=dimensionboundaries[i][0]

In [19]:
class PSO():
    def __init__(self,costFunc,num_dimensions,dimensionboundaries,num_particles,maxiter):
        iterationanimation=[]
        particlelocationsanimation=[]
        erroranimation=[]
        error_best_swarm=-1                   # best error for group
        pos_best_swarm=[]                   # best position for group

        # establish the swarm
        swarm=[]
        startlocation=[]
        for i in range(0,num_particles):
            #each particle in the swarm starts from random starting location in the search space
            startlocation=[random.uniform(-0.99999*resolution,0.99999*resolution),random.uniform(-0.99999*resolution,0.99999*resolution)]
#             print(startlocation)
            swarm.append(Particle(startlocation))

        # begin optimization loop
        i=0
        result['MaxIterationsToConverge']=maxiter
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].error_particle < error_best_swarm or error_best_swarm == -1:
                    pos_best_swarm=list(swarm[j].loc_particle)
                    error_best_swarm=float(swarm[j].error_particle)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_swarm)
                swarm[j].update_position(dimensionboundaries)
                
###START OF SNIPPET: DATA GATHERING FOR ANIMATION####
            swarmlocn=[]   
            for partnum in range(num_particles):
                templist=[list(swarm[partnum].loc_particle)[0],list(swarm[partnum].loc_particle)[1]]
                swarmlocn.append(templist)
            iterationanimation.append(i)
            particlelocationsanimation.append(swarmlocn)
            erroranimation.append(error_best_swarm)
###END OF SNIPPET: DATA GATHERING FOR ANIMATION####

            if error_best_swarm<0.001:
                result['MaxIterationsToConverge']=i
                result['NumParticles']=num_particles
                result['EstimatedGlobalMinima']=pos_best_swarm
                result['MinFunctionCost']=error_best_swarm
                break
            i+=1
            
###START OF SNIPPET: DATA GATHERING FOR ANIMATION####
        d={'Iteration':iterationanimation,'ParticleLocations':particlelocationsanimation,'FunctionCost':erroranimation}
        df=pd.DataFrame()
        df=pd.DataFrame(d,columns=columns)
        df.to_csv('PSO_Rosenbrock_'+str(num_particles)+('particles.csv'),index=False,header=True)
        print(df.shape)
###END OF SNIPPET: DATA GATHERING FOR ANIMATION####
        # save final results
        result['NumParticles']=num_particles
        result['EstimatedGlobalMinima']=pos_best_swarm
        result['MinFunctionCost']=error_best_swarm
        print(result)

In [20]:
if __name__ == "__PSO__":
    main()

#--- RUN ----------------------------------------------------------------------+
global num_dimensions
num_dimensions = 2
global resolution
resolution=20
dimensionboundaries=[]
for dimension in range(num_dimensions):
    DimBoundary=(-resolution,resolution)
    dimensionboundaries.append(DimBoundary)
# print(dimensionboundaries)
global result
result={'Function':'rosenbrockCostFunc',
        'NumParticles':5,
        'MaxIterationsToConverge':[],
        'EstimatedGlobalMinima':[],
        'MinFunctionCost':[]}
for particletrials in range(6):
    PSO(func1,num_dimensions=num_dimensions,dimensionboundaries=dimensionboundaries,num_particles=5*(1+particletrials),maxiter=1000)

(1000, 3)
{'Function': 'rosenbrockCostFunc', 'NumParticles': 5, 'MaxIterationsToConverge': 1000, 'EstimatedGlobalMinima': [4.4715769032608135, 20.0], 'MinFunctionCost': 19.9975}
(1000, 3)
{'Function': 'rosenbrockCostFunc', 'NumParticles': 10, 'MaxIterationsToConverge': 1000, 'EstimatedGlobalMinima': [4.471576902998923, 20.0], 'MinFunctionCost': 19.9975}
(56, 3)
{'Function': 'rosenbrockCostFunc', 'NumParticles': 15, 'MaxIterationsToConverge': 55, 'EstimatedGlobalMinima': [-0.012769544700512822, 0.00031211676160650267], 'MinFunctionCost': 0.00016528302576079992}
(52, 3)
{'Function': 'rosenbrockCostFunc', 'NumParticles': 20, 'MaxIterationsToConverge': 51, 'EstimatedGlobalMinima': [0.014381016100946567, -0.0013070911489189152], 'MinFunctionCost': 0.000436004390271323}
(53, 3)
{'Function': 'rosenbrockCostFunc', 'NumParticles': 25, 'MaxIterationsToConverge': 52, 'EstimatedGlobalMinima': [-0.009406986212787911, 0.00015691732752054835], 'MinFunctionCost': 8.895960050550876e-05}
(38, 3)
{'Funct

In [12]:
## generating rosenbrock cost to visualize
# rosenbrockcost=[]
# for i in range(0,10):
#     for j in range (0,10):
#         rosenbrockcost.append(i**2 + 100*(j-i**2)**2)
#     print(rosenbrockcost[10*i:10*(i+1)])
#         print('rosenbrock cost at '+str(i)+', '+str(j)+':'+ str(i**2 + 100*(j-i**2)**2))


In [144]:
# # generating rastrigin cost to visualize
# rastrigincost=[]
# for i in range(0,10):
#     for j in range (0,10):
#         rastrigincost.append(10*len(initiallocation)+(i/2)**2-10*(math.cos(2*math.pi*i/2))+(j/2)**2-10*(math.cos(2*math.pi*j/2)))
#     print(rastrigincost[10*i:10*(i+1)])



In [149]:
x = np.linspace(dimensionboundaries[0][0], dimensionboundaries[0][1], 50)
print(x)
# y = np.linspace(dimensionboundaries[1][0], dimensionboundaries[1][1], 50)
# X, Y = np.meshgrid(x, y)
# # features=[[x,y] for x,y in zip(X,Y)]
# Z = np.array([func1(features) for x, y in zip(X, Y)])
# # initialize figure
# fig = plt.figure(figsize=(13, 6))
# ax1 = fig.add_subplot(121, facecolor='w')
# ax2 = fig.add_subplot(122, facecolor='w')
# plt.show()

[-20.         -19.18367347 -18.36734694 -17.55102041 -16.73469388
 -15.91836735 -15.10204082 -14.28571429 -13.46938776 -12.65306122
 -11.83673469 -11.02040816 -10.20408163  -9.3877551   -8.57142857
  -7.75510204  -6.93877551  -6.12244898  -5.30612245  -4.48979592
  -3.67346939  -2.85714286  -2.04081633  -1.2244898   -0.40816327
   0.40816327   1.2244898    2.04081633   2.85714286   3.67346939
   4.48979592   5.30612245   6.12244898   6.93877551   7.75510204
   8.57142857   9.3877551   10.20408163  11.02040816  11.83673469
  12.65306122  13.46938776  14.28571429  15.10204082  15.91836735
  16.73469388  17.55102041  18.36734694  19.18367347  20.        ]


In [ ]:
def visualizeHistory2D(func=None, history=None, bounds=None, 
                       minima=None, func_name='', save2mp4=False, save2gif=False):
    print('## Visualizing optimizing {}'.format(func_name))
    assert len(bounds)==2
    # define meshgrid according to given boundaries
    x = np.linspace(bounds[0][0], bounds[0][1], 50)
    y = np.linspace(bounds[1][0], bounds[1][1], 50)
    X, Y = np.meshgrid(x, y)
    Z = np.array([func([x, y]) for x, y in zip(X, Y)])

    # initialize figure
    fig = plt.figure(figsize=(13, 6))
    ax1 = fig.add_subplot(121, facecolor='w')
    ax2 = fig.add_subplot(122, facecolor='w')

    # animation callback function
    def animate(frame, history):
        # print('current frame:',frame)
        ax1.cla()
        ax1.set_xlabel('X1')
        ax1.set_ylabel('X2')
        ax1.set_title('{}|iter={}|Gbest=({:.5f},{:.5f})'.format(func_name,frame+1,
                      history['global_best'][frame][0], history['global_best'][frame][1]))
        ax1.set_xlim(bounds[0][0], bounds[0][1])
        ax1.set_ylim(bounds[1][0], bounds[1][1])
        ax2.set_xlabel('Iteration')
        ax2.set_ylabel('Fitness')
        ax2.set_title('Minima Value Plot|Population={}|MinVal={:}'.format(len(history['particles'][0]),history['global_best_fitness'][frame]))
        ax2.set_xlim(2,len(history['global_best_fitness']))
        ax2.set_ylim(10e-16,10e0)
        ax2.set_yscale('log')

        # data to be plot
        data = history['particles'][frame]
        global_best = np.array(history['global_best_fitness'])

        # contour and global minimum
        contour = ax1.contour(X,Y,Z, levels=50, cmap="magma")
        ax1.plot(minima[0], minima[1] ,marker='o', color='black')

        # plot particles
        ax1.scatter(data[:,0], data[:,1], marker='x', color='black')
        if frame > 1:
            for i in range(len(data)):
                ax1.plot([history['particles'][frame-n][i][0] for n in range(2,-1,-1)],
                         [history['particles'][frame-n][i][1] for n in range(2,-1,-1)])
        elif frame == 1:
            for i in range(len(data)):
                ax1.plot([history['particles'][frame-n][i][0] for n in range(1,-1,-1)],
                         [history['particles'][frame-n][i][1] for n in range(1,-1,-1)])

        # plot current global best
        x_range = np.arange(1, frame+2)
        ax2.plot(x_range, global_best[0:frame+1])
        
    # title of figure
    fig.suptitle('Optimizing of {} function by PSO, f_min({},{})={}'.format(func_name.split()[0],
                                                                      minima[0],minima[1],
                                                                      func(minima)),fontsize=20)

    ani = animation.FuncAnimation(fig, animate, fargs=(history,),
                    frames=len(history['particles']), interval=250, repeat=False, blit=False)

    ## TODO: Save animation as mp4
    if save2mp4:
        os.makedirs('mp4/', exist_ok=True)
        ani.save('mp4/PSO_{}_population_{}.mp4'.format(func_name.split()[0], len(history['particles'][0])), writer="ffmpeg", dpi=100)
        print('A mp4 video is saved at mp4/')

    plt.show()
